In [1]:
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder

import pandas as pd
import numpy
import tensorflow as tf
import matplotlib.pyplot as plt 
import seaborn as sns 

# seed 값 설정
seed=0
numpy.random.seed(3)
tf.random.set_seed(3)

Using TensorFlow backend.


In [2]:
# 데이터 불러오기
df = pd.read_csv('data/train.csv')
df.head(10)

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales
0,1,CA-2017-152156,08/11/2017,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600
1,2,CA-2017-152156,08/11/2017,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400
2,3,CA-2017-138688,12/06/2017,16/06/2017,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036.0,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200
3,4,US-2016-108966,11/10/2016,18/10/2016,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775
4,5,US-2016-108966,11/10/2016,18/10/2016,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680
5,6,CA-2015-115812,09/06/2015,14/06/2015,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,California,90032.0,West,FUR-FU-10001487,Furniture,Furnishings,Eldon Expressions Wood and Plastic Desk Access...,48.8600
6,7,CA-2015-115812,09/06/2015,14/06/2015,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,California,90032.0,West,OFF-AR-10002833,Office Supplies,Art,Newell 322,7.2800
7,8,CA-2015-115812,09/06/2015,14/06/2015,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,California,90032.0,West,TEC-PH-10002275,Technology,Phones,Mitel 5320 IP Phone VoIP phone,907.1520
8,9,CA-2015-115812,09/06/2015,14/06/2015,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,California,90032.0,West,OFF-BI-10003910,Office Supplies,Binders,DXL Angle-View Binders with Locking Rings by S...,18.5040
9,10,CA-2015-115812,09/06/2015,14/06/2015,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,California,90032.0,West,OFF-AP-10002892,Office Supplies,Appliances,Belkin F5C206VTEL 6 Outlet Surge,114.9000


In [3]:
df.info()
# object 타입이 많아 엑셀을 통해 csv 파일을 직접 확인했음.
# object 타입을 범주형으로 변환해야할 타입이 많음, 날짜 데이터는 년, 월, 일을 분리하여 새로운 행으로 생성
# Order ID, Row ID, Customer ID, customer ID는 고객과 주문을 식별하기 위한 컬럼이므로 판매량 예측과는 상관이 없음.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9800 entries, 0 to 9799
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Row ID         9800 non-null   int64  
 1   Order ID       9800 non-null   object 
 2   Order Date     9800 non-null   object 
 3   Ship Date      9800 non-null   object 
 4   Ship Mode      9800 non-null   object 
 5   Customer ID    9800 non-null   object 
 6   Customer Name  9800 non-null   object 
 7   Segment        9800 non-null   object 
 8   Country        9800 non-null   object 
 9   City           9800 non-null   object 
 10  State          9800 non-null   object 
 11  Postal Code    9789 non-null   float64
 12  Region         9800 non-null   object 
 13  Product ID     9800 non-null   object 
 14  Category       9800 non-null   object 
 15  Sub-Category   9800 non-null   object 
 16  Product Name   9800 non-null   object 
 17  Sales          9800 non-null   float64
dtypes: float

In [7]:
def date(df, column):
    df[column] = pd.to_datetime(df['Ship Date'])
    df[column] = df[column+'Date'].apply(lambda x : x.year)
    df[column] = df[column+'Ship Date'].apply(lambda x : x.month)
    df[column] = df[column+'Ship Date'].apply(lambda x : x.day)
    df = df.drop(column, axis=1)
    return df

In [10]:
def oneHotEncoding(df, column):
    dummy = pd.get_dummies(df[column], prefix=column)
    df = pd.concat([df, dummy], axis=1)
    df = df.drop(column, axis=1)
    return df

In [11]:
date(df, 'Order Date')
date(df, 'Ship Date')

KeyError: 'Ship Date'

In [13]:
oneHotEncoding(df, 'Category')

,Row ID,Order ID,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,...,Sales,Order_year,Order_month,Order_day,Ship_year,Ship_month,Ship_day,Category_Furniture,Category_Office Supplies,Category_Technology
0,1,CA-2017-152156,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,...,261.9600,2017,8,11,2017,11,11,1,0,0
1,2,CA-2017-152156,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,...,731.9400,2017,8,11,2017,11,11,1,0,0
2,3,CA-2017-138688,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036.0,...,14.6200,2017,12,6,2017,6,16,0,1,0
3,4,US-2016-108966,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311.0,...,957.5775,2016,11,10,2016,10,18,1,0,0
4,5,US-2016-108966,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311.0,...,22.3680,2016,11,10,2016,10,18,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9795,9796,CA-2017-125920,Standard Class,SH-19975,Sally Hughsby,Corporate,United States,Chicago,Illinois,60610.0,...,3.7980,2017,5,21,2017,5,28,0,1,0
9796,9797,CA-2016-128608,Standard Class,CS-12490,Cindy Schnelling,Corporate,United States,Toledo,Ohio,43615.0,...,10.3680,2016,12,1,2016,1,17,0,1,0
9797,9798,CA-2016-128608,Standard Class,CS-12490,Cindy Schnelling,Corporate,United States,Toledo,Ohio,43615.0,...,235.1880,2016,12,1,2016,1,17,0,0,1
9798,9799,CA-2016-128608,Standard Class,CS-12490,Cindy Schnelling,Corporate,United States,Toledo,Ohio,43615.0,...,26.3760,2016,12,1,2016,1,17,0,0,1


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9800 entries, 0 to 9799
Data columns (total 22 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   Row ID         9800 non-null   int64   
 1   Order ID       9800 non-null   object  
 2   Ship Mode      9800 non-null   object  
 3   Customer ID    9800 non-null   object  
 4   Customer Name  9800 non-null   object  
 5   Segment        9800 non-null   object  
 6   Country        9800 non-null   object  
 7   City           9800 non-null   object  
 8   State          9800 non-null   object  
 9   Postal Code    9789 non-null   float64 
 10  Region         9800 non-null   object  
 11  Product ID     9800 non-null   object  
 12  Category       9800 non-null   category
 13  Sub-Category   9800 non-null   object  
 14  Product Name   9800 non-null   object  
 15  Sales          9800 non-null   float64 
 16  Order_year     9800 non-null   int64   
 17  Order_month    9800 non-null   in

0       1.0
1       1.0
2       0.0
3       1.0
4       0.0
       ... 
9795    0.0
9796    0.0
9797    0.0
9798    0.0
9799    0.0
Name: Category, Length: 9800, dtype: float32

In [17]:
df.describe()

,Row ID,Postal Code,Category,Sales,Order_year,Order_month,Order_day,Ship_year,Ship_month,Ship_day
count,9800.000000,9789.000000,9800.000000,9800.000000,9800.000000,9800.000000,9800.000000,9800.000000,9800.000000,9800.000000
mean,4900.500000,55273.322403,0.212041,230.769059,2016.724184,7.186122,16.119184,2016.739388,7.299082,16.348980
std,2829.160653,32041.223413,0.408774,626.651875,1.123984,3.424108,8.073122,1.126837,3.387610,8.330751
min,1.000000,1040.000000,0.000000,0.444000,2015.000000,1.000000,1.000000,2015.000000,1.000000,1.000000
25%,2450.750000,23223.000000,0.000000,17.248000,2016.000000,4.000000,10.000000,2016.000000,5.000000,10.000000
50%,4900.500000,58103.000000,0.000000,54.490000,2017.000000,8.000000,16.000000,2017.000000,8.000000,16.000000
75%,7350.250000,90008.000000,0.000000,210.605000,2018.000000,10.000000,23.000000,2018.000000,10.000000,24.000000
max,9800.000000,99301.000000,1.000000,22638.480000,2018.000000,12.000000,31.000000,2019.000000,12.000000,31.000000


1

In [5]:
df.corr()

,Row ID,Postal Code,Sales
Row ID,1.000000,0.013645,0.001151
Postal Code,0.013645,1.000000,-0.024067
Sales,0.001151,-0.024067,1.000000
